<a href="https://colab.research.google.com/github/PlumNoseBear/PAINN/blob/main/_aiforfinance2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задание хакатона AI for Finance Hack 2025 предварительный этап
Задача здесь:
https://drive.google.com/drive/folders/1tQZ9F27InAtt8oGwmLvZY9hMXPxSLpJv

устанавливаем разрешенные бибилиотеки

In [ ]:
!pip install faiss-cpu==1.11.0
!pip install langchain_community
!pip install anyio

импорт библиотек

In [ ]:
import pandas as pd
import openai
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from openai import OpenAI
from tqdm import tqdm
import re
import numpy as np
import faiss
from google.colab import userdata
from tiktoken import get_encoding
import anyio
from pathlib import Path
import os
import json
from typing import List, Dict, Optional

грузим данные

In [ ]:
!curl -L "https://drive.google.com/uc?export=download&id=1Q2UMOYiPdvP5YH4aTnbJxbmVokJu2hem&confirm=t" -o train_data.csv
!curl -L "https://drive.google.com/uc?export=download&id=1YBtkFqvfV4Hcq6c2SikC1HM4B4iIVlSh&confirm=t" -o questions.csv

df = pd.read_csv("train_data.csv")
uniq_tags = set()
for tag_str in df["tags"]:
    # Извлекаем все слова в кавычках
    matches = re.findall(r"'(.*?)'", tag_str)
    uniq_tags.update(matches)

# Преобразуем в отсортированный список
uniq_tags = sorted(uniq_tags)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5819k  100 5819k    0     0  3317k      0  0:00:01  0:00:01 --:--:-- 18.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75610  100 75610    0     0  47205      0  0:00:01  0:00:01 --:--:--  473k


Промты

In [ ]:
system_promt = f"""
Ты — AI-ассистент технической поддержки для финансовой организации, работающий в составе мультиагентной системы (гыук_зкщье).

**Основные принципы работы:**

1. **Безопасность и конфиденциальность:**
   - Никогда не запрашивай и не храни личные данные клиентов (паспортные данные, номера карт, CVV, PIN-коды, пароли).
   - Не выполняй операции, требующие ручной обработки (например, изменение лимитов по карте, выдача кредитов).

2. **Регуляторные требования:**
   - Не давай финансовых или инвестиционных советов. Перенаправляй такие запросы к лицензированным специалистам.
   - Всегда информируй клиента о возможных комиссиях, рисках и условиях услуг.
   - Твоя задача — только отвечать на вопросы, ничего не придумывая от себя.

3. **Точность и прозрачность:**
   - Если не знаешь ответ, честно признай это и предложи уточнить информацию у специалиста или найти решение в базе знаний.
   - Избегай двусмысленных формулировок. Давай четкие и структурированные инструкции.
   - Используй только теги из {uniq_tags}. Приводи все теги к точному соответствию значениям из {uniq_tags}.

4. **Клиентоориентированность:**
   - Используй вежливый и профессиональный тон.
   - Адаптируй ответы под уровень технической грамотности клиента.
   - Предлагай альтернативные решения, если основной запрос не может быть выполнен.

**Функциональные обязанности:**

- **Техническая поддержка:**
  - Помощь клиентам с вопросами на основании доступной внутренней базы данных.
  - Решение проблем на основании данных из внутренней базы.
  - Консультации по финансовым вопросам в пределах доступной внутренней базы данных.
  - Консультирование о мошеннических действиях (блокировка карт, проверка транзакций).

- **Эскалация:**
  - Уведомляй клиента о необходимости передачи сложных случаев в соответствующие отделы.

**Ограничения:**
- Не интерпретируй юридические или налоговые вопросы. Перенаправляй к юристам или налоговым консультантам.
- Не обещай сроки решения проблем, если они зависят от третьих сторон (например, банков-партнеров).

**Формат ответов:**
- Всегда начинай сгенерированный ответ клиенту с подстроки '<ОТВЕТ:>', а в конце ответа ставь '<>'.
- Следуй структуре, заданной в user_prompt для каждого агента.

**Тон и стиль общения:**
- Профессиональный, но дружелюбный: используй обращения по имени, если клиент представился.
- Краткость и ясность: избегай сложных терминов. Объясняй простыми и понятными словами.
- Эмпатия: показывай понимание проблемы клиента (например: "Я понимаю, как это может быть неприятно. Давайте решим вопрос вместе").

**Исключительные ситуации:**
- Если клиент проявляет агрессию: сохраняй спокойствие, предлагай помощь и при необходимости передавай разговор оператору.
- Если клиент находится в стрессе (например, из-за мошенничества): удели дополнительное время на объяснения и подтверждение каждого шага.
"""

In [ ]:
user_promt = f"""
Мультиагентная система для технической поддержки банка.

Структура работы:
1. Агент 1: Pre-Processor (Аналитик Запроса)
   - Задача: Классифицировать запрос клиента, составить еще 1 близких по смыслу запроса клиента, выделить ключевые сущности их всех трех запросов и подобрать к ним теги из {uniq_tags}.
   - Формат вывода:
     Запрос клиента: [текст запроса]
     Тип запроса: [FAQ/Техническая проблема/Финансовый запрос/Консультация]
     Ключевые сущности: [список сущностей]
     Теги: [список тегов из {uniq_tags}]
     Контекст: [релевантные данные из {df['text']}]

2. Агент 2: Embedding Searcher (Поисковик по Эмбеддингам)
   - Задача: Найти топ-3 релевантных ответа по эмбеддингам, используя теги {uniq_tags}.
   - Формат вывода:
     Релевантные ответы:
     1. [Ответ 1] (Доверие: [процент])
        Теги: [список тегов]
     2. [Ответ 2] (Доверие: [процент])
        Теги: [список тегов]
     3. [Ответ 3] (Доверие: [процент])
        Теги: [список тегов]

3. Агент 3: Response Generator (Генератор Ответов)
Использовать LLM для генерации точного и понятного ответа клиенту на основе данных,
полученных от Knowledge Retriever. Response Generator должен:
Сгенерировать ответ на запрос клиента с учетом:Тона и стиля общения банка.Использовать в
ответе только источники с релевантностью выше 75%. Выдать по предоставленному контексту с
источниками ответ клиенту, используя в первую очередь источники с большей релевантностью.
Если источники содержат противоречия, то  источник с большей релевантностью включить в ответ,
а с меньшей оставить как примечание к основному ответу.
Подготовить точный и понятный ответ для клиента.
Формат вывода:<ОТВЕТ:>'[Текст ответа]'<>'.

4. Агент 4: Post-Processor (Обработчик Результатов)
   - Задача: Проверить ответ и доставить его клиенту. Если ответ "<ОТВЕТ:>Нет ответа<>", запустить повторный анализ с Агента 1.
   - Формат вывода:
     Статус: [Ответ доставлен/Повторный анализ]

ООБЯЗАТЕЛЬНЫЕ правила:
- Последовательность выполнения: Агент 1 → Агент 2 → Агент 3 → Агент 4.
- Если ответ не найден, система возвращается к Агенту 1 для повторного анализа ЕЩЕ 3 раза. ТОЛЬКО ПОСЛЕ ЭТОГО выдается "<ОТВЕТ:>Нет ответа<>".
- Используются только теги из {uniq_tags}.
- Ответы клиенту начинаются с "<ОТВЕТ:>" и заканчиваются "<>".
"""


In [ ]:
# @title Выбор модели LLM и эмбеддингов
output_file = 'submission.csv'  # @param {type:"string"}
# @markdown ### Выбор модели LLM и эмбеддингов
model_name = "openrouter/mistralai/mistral-small-3.2-24b-instruct"  # @param ["openrouter/mistralai/mistral-small-3.2-24b-instruct", "openrouter/meta-llama/llama-3-70b-instruct", "openrouter/x-ai/grok-3-mini", "openrouter/google/gemma-3-27b-it"] {type:"string"}
model_embed = "text-embedding-3-small"  # @param ["text-embedding-3-small", "text-embedding-ada-002"] {type:"string"}

In [ ]:
# Пути к файлам
QUESTIONS_CSV = "questions.csv"
SUBMISSION_CSV = "submission.csv"
STATE_FILE = "state.json"  # Файл для сохранения состояния (индекс последнего обработанного вопроса)


# Функция для разбиения текста на чанки
def split_text(text: str, max_tokens: int = 8191) -> List[str]:
    encoding = get_encoding("p50k_base")
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(encoding.decode(chunk))
    return chunks

# Асинхронная функция для получения эмбеддингов
async def get_embeddings(texts: List[str], embedding_model_name: str) -> List[List[float]]:
    client = openai.AsyncOpenAI(
    base_url="https://ai-for-finance-hack.up.railway.app/",
    api_key = userdata.get('TEXT_EMBER')
    )
    all_embeddings = []
    # Пакет из 20 блоков * ~ 8191 токен/блок = ~160 тыс. токенов, чтобы уложиться в лимит с запасом.
    batch_size = 20

    for i in tqdm(range(0, len(texts), batch_size), desc="Getting embeddings in batches"):
        batch_texts = texts[i:i + batch_size]
        try:
            response = await client.embeddings.create(
                model=embedding_model_name,
                input=batch_texts,
            )
            all_embeddings.extend([item.embedding for item in response.data])
        except Exception as e:
            print(f"Ошибка при получении эмбеддингов для батча: {e}")
            return []
    return all_embeddings

In [ ]:
embeddings_model = OpenAIEmbeddings(
    openai_api_base="https://ai-for-finance-hack.up.railway.app/",
    openai_api_key=userdata.get('TEXT_EMBER'),
    model=model_embed,
    chunk_size=100 # Выбираем  размер блока, чтобы соблюсти ограничение на токены API
)

# Загружаем данные для контекста
df = pd.read_csv("train_data.csv")

faiss_index_dir = "faiss_index_storage" # Каталог для LangChain FAISS index

# Загружаем или создаём FAISS-индекс
if Path(faiss_index_dir).exists(): # проверяем наличие католога
    print("Грузим LangChain FAISS index...")
    db = FAISS.load_local(
        faiss_index_dir,
        embeddings_model,
        index_name="faiss_index",
        allow_dangerous_deserialization=True
    )
    index = db.index
    chunked_texts = [doc.page_content for doc in db.docstore._dict.values()] # Добавляем чанки

else:
    print("Создаем LangChain FAISS index...")
    # Получаем эмбеддинги для всех текстов в df
    all_texts = df["text"].tolist()
    documents = []
    chunked_texts = [] # Список для чанков
    for i, text in enumerate(all_texts):
        # Разбиваем тексты а чанки
        chunks = split_text(text, max_tokens=8191)
        chunked_texts.extend(chunks) # Добавляем чанки в список чанков
        for j, chunk in enumerate(chunks):
            # Создаем  объект документа для чанков
            documents.append(Document(page_content=chunk, metadata={"source": f"doc_{i}_chunk_{j}"}))

    # Создание векторного хранилища FAISS на основе документов
    db = FAISS.from_documents(documents, embeddings_model)
    # Сохраняем индекс FAISS
    db.save_local(faiss_index_dir, index_name="faiss_index")

    index = db.index

In [ ]:
# Асинхронная функция для обработки одного вопроса
async def process_question(
    question: str,
    index: faiss.Index,
    df: pd.DataFrame,
    chunked_texts: List[str], # Добавляем chunked_texts
    k: int = 4
) -> Optional[str]:
    # Инициализация клиента OpenAI
    chat_client = openai.AsyncOpenAI(
        base_url="https://ai-for-finance-hack.up.railway.app/",
        api_key = userdata.get('OpenR-LLM')
    )
    try:
        # Поиск релевантных ответов
        query_embedding = await get_embeddings([question], model_embed)
        if not query_embedding:
            return None

        distances, indices = index.search(np.array(query_embedding), k)
        # Извлекем тексты, по индексам из chunked_texts
        relevant_texts = [chunked_texts[idx] for idx in indices[0]]

        # Генерация ответа
        promt = f"""
        Контекст:
        {" ".join(relevant_texts)}
        Вопрос: {question}
        Ответь на вопрос клиента, используя только предоставленный контекст.
        Начни ответ с <ОТВЕТ:> и заверши <>.ЕМБЕДДИНГ
        """
        response = await chat_client.chat.completions.create(
            model=model,
            messages=[
                {   "role": "system","content":f"{system_promt}",
                    "role": "user",
                    "content": [{"type": "text",
                                 "text": f"{user_promt}: {promt}"
                                }
                               ]
                }
                     ],
            seed=3111696,
            temperature=0.25,
            max_tokens=8191
        )
        answer = response.choices[0].message.content
        return answer
    except Exception as e:
        print(f"Ошибка при обработке вопроса: {e}")
        return None

# Асинхронная функция для записи ответа в submission.csv
async def write_answer(
    question_id: int,
    question: str,
    answer: str,
    submission_file: str,
    mode: str = "a"
) -> None:
    data = {"id": question_id, "question": question, "answer": answer}
    df = pd.DataFrame([data])
    header = not Path(submission_file).exists() or mode == "w"
    df.to_csv(submission_file, mode=mode, index=False, header=header, encoding="utf-8")

Обработка всего массива вопросов с продолжением с места окончания в случае прерывания

In [ ]:
# Асинхронная функция для обработки всех вопросов
async def process_questions(
    questions_file: str,
    submission_file: str,
    state_file: str,
    index: faiss.Index,
    df: pd.DataFrame,
    chunked_texts: List[str]
) -> None:
    # Загружаем вопросы
    questions_df = pd.read_csv(questions_file)

    # Загружаем состояние (индекс последнего обработанного вопроса)
    last_processed_index = 0
    if Path(state_file).exists():
        with open(state_file, "r") as f:
            state = json.load(f)
            last_processed_index = state.get("last_processed_index", 0)

    # Обрабатываем вопросы, начиная с последнего сохранённого индекса
    for idx, row in questions_df.iterrows():
        if idx < last_processed_index:
            continue

        question = row["Вопрос"]
        question_id = row.get("id", idx)

        print(f"Обработка вопроса {idx}: {question}")

        # Обрабатываем вопрос
        answer = await process_question(question, index, df, chunked_texts) # Pass chunked_texts
        if answer:

            await write_answer(question_id, question, answer, submission_file)

            # Сохраняем состояние
            with open(state_file, "w") as f:
                json.dump({"last_processed_index": idx}, f)

# Основная асинхронная функция
async def main():

    # Обрабатываем вопросы
    await process_questions(QUESTIONS_CSV, SUBMISSION_CSV, STATE_FILE, index, df, chunked_texts)


Запуск  программы с обработкой всех вопросов из questions.scv

In [ ]:
await main()

Поиск соответствий в базе

In [ ]:
# @title ### Введите вопрос:
query = 'Какие риски инвестиций в криптовалюту'  # @param {type:"string"}


In [ ]:
docs_and_scores = db.similarity_search_with_score(query, k=4)

# Вывод результатов
for doc, score in docs_and_scores:
    print(f"Score: {score}")
    print(f"Text: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("---")


In [ ]:
# Ответ на вопрос
await process_question(query,index, df,chunked_texts, 4)


Getting embeddings in batches: 100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


'<ОТВЕТ:>\nИнвестиции в криптовалюту связаны с рядом значительных рисков, которые важно учитывать:\n\n1. **Высокий волатильность**: Цены на криптовалюты могут резко меняться, что делает их инвестиции особенно рискованными. Это отличает их от более стабильных активов, таких как облигации или вклады.\n\n2. **Отсутствие государственной страховки**: В отличие от банковских вкладов, инвестиции в криптовалюту не защищены системой страхования вкладов. В случае потери средств (например, из-за взлома кошелька или банкротства биржи) вернуть их будет сложно.\n\n3. **Риск мошенничества**: Криптовалютный рынок привлекает мошенников, предлагающих сомнительные проекты, пирамиды или фишинговые схемы. Важно внимательно проверять любые предложения.\n\n4. **Регуляторные риски**: В разных странах законодательство о криптовалютах может меняться, что влияет на их ликвидность и доступность. Например, в некоторых юрисдикциях могут вводиться ограничения или запреты.\n\n5. **Технические риски**: Потеря доступа 